In [1]:
# Import libraries
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import h3
from scipy.sparse import lil_matrix
from torch_geometric.utils import from_scipy_sparse_matrix
from scipy.sparse import csr_matrix
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader, Dataset


In [2]:
# Load the data
data = pd.read_csv('dataset/sales_4.csv')

# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Selec top N Stores
top_n = 100

# Calculate lifetime product count for each store (sum of product_count across all weeks)
store_lifetime_product_count = data.groupby('store_id')['product_count'].sum()

# Sort stores by lifetime product_count in descending order and get the top n
top_n_stores = store_lifetime_product_count.sort_values(ascending=False).head(top_n).index

# Filter the original data to include only the top n stores
filtered_data = data[data['store_id'].isin(top_n_stores)]

# Debug: Print the filtered data and its shape
print("Filtered data for top n stores:")
print(filtered_data.head(5))  # Display the first 5 rows of filtered data

print("Shape of filtered data:")
print(filtered_data.shape)  # Display the shape of the filtered data


Filtered data for top n stores:
        date locality_type product_name  product_count   latitude   longitude  \
1 2021-01-01       Diamond         Beta              1  40.858416  -73.781928   
2 2021-01-01       Diamond        Alpha              1  33.363745 -118.424787   
4 2021-01-01       Diamond        Alpha              1  30.769735  -91.458505   
5 2021-01-01       Diamond        Gamma              1  41.274267  -81.993373   
6 2021-01-01       Diamond        Delta              1  29.967208  -97.319137   

   store_id  
1         2  
2         3  
4         5  
5         6  
6         7  
Shape of filtered data:
(6401132, 7)


In [3]:

# Aggregate data to weekly demand per store
filtered_data['week'] = filtered_data['date'].dt.to_period('W').apply(lambda r: r.start_time)

# Ensure there are no duplicate entries for store_id and week
weekly_data = filtered_data.groupby(['store_id', 'week']).agg({
    'product_count': 'sum',  # Sum demand for duplicate entries
    'latitude': 'first',    # Take the first latitude (since it's the same for each store_id)
    'longitude': 'first',   # Take the first longitude
    'locality_type': 'first'  # Take the first locality_type
}).reset_index()

# Debug: Print the shape of the aggregated data
print("Shape of weekly_data after aggregation:")
print(weekly_data.shape)

# Normalize demand
weekly_data['product_count'] = (weekly_data['product_count'] - weekly_data['product_count'].mean()) / weekly_data['product_count'].std()

# Debug: Print the first few rows of normalized data
print("First few rows of normalized weekly_data:")
print(weekly_data.head(5))

# Encode locality type
locality_mapping = {'Diamond': 0, 'Gold': 1, 'Silver': 2}
weekly_data['locality_type'] = weekly_data['locality_type'].map(locality_mapping)

# Debug: Print the first few rows after encoding locality type
print("First few rows after encoding locality_type:")
print(weekly_data.head(5))

# Pivot the data to create weekly time-series input
time_series = weekly_data.pivot(index='store_id', columns='week', values='product_count').fillna(0)

# Debug: Print the shape of the time_series
print(f"Shape of time_series: {time_series.shape}")


C:\Users\rznis\AppData\Local\Temp\ipykernel_10404\440618532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['week'] = filtered_data['date'].dt.to_period('W').apply(lambda r: r.start_time)


Shape of weekly_data after aggregation:
(20448, 6)
First few rows of normalized weekly_data:
   store_id       week  product_count   latitude  longitude locality_type
0         2 2020-12-28      -0.916058  40.858416 -73.781928       Diamond
1         2 2021-01-04       3.803672  40.858416 -73.781928       Diamond
2         2 2021-01-11       4.115463  40.858416 -73.781928       Diamond
3         2 2021-01-18       3.834851  40.858416 -73.781928       Diamond
4         2 2021-01-25       3.760801  40.858416 -73.781928       Diamond
First few rows after encoding locality_type:
   store_id       week  product_count   latitude  longitude  locality_type
0         2 2020-12-28      -0.916058  40.858416 -73.781928              0
1         2 2021-01-04       3.803672  40.858416 -73.781928              0
2         2 2021-01-11       4.115463  40.858416 -73.781928              0
3         2 2021-01-18       3.834851  40.858416 -73.781928              0
4         2 2021-01-25       3.760801  40.8

In [4]:
# Replace Haversine with H3 Indexing
# Add H3 cells to the dataframe
resolution = 3
weekly_data['h3_index'] = weekly_data.apply(
    lambda row: h3.latlng_to_cell(row['latitude'], row['longitude'], resolution), axis=1
)

# Group by H3 cells
h3_to_store_map = weekly_data.groupby('h3_index')['store_id'].apply(list).to_dict()

# Find neighboring H3 cells
h3_neighbors = {h: h3.grid_disk(h, 1) for h in h3_to_store_map.keys()}

# Create adjacency matrix
num_stores = len(weekly_data['store_id'].unique())
adj_matrix = lil_matrix((num_stores, num_stores))

store_to_idx = {store_id: idx for idx, store_id in enumerate(weekly_data['store_id'].unique())}

for h3_index, neighbors in h3_neighbors.items():
    stores_in_hex = h3_to_store_map.get(h3_index, [])
    for neighbor in neighbors:
        neighbor_stores = h3_to_store_map.get(neighbor, [])
        for s1 in stores_in_hex:
            for s2 in neighbor_stores:
                idx1 = store_to_idx[s1]
                idx2 = store_to_idx[s2]
                adj_matrix[idx1, idx2] = 1  # Create an edge

# Debug: Print the shape of adjacency matrix
print(f"Shape of adjacency matrix: {adj_matrix.shape}")


Shape of adjacency matrix: (100, 100)


In [5]:
# Convert adjacency matrix to sparse format and PyTorch Geometric format
adj_sparse = adj_matrix.tocsr()
edge_index, edge_weight = from_scipy_sparse_matrix(adj_sparse)

# Debug: Print the shapes of edge_index and edge_weight
print(f"Shape of edge_index: {edge_index.shape}")
print(f"Shape of edge_weight: {edge_weight.shape}")


Shape of edge_index: torch.Size([2, 294])
Shape of edge_weight: torch.Size([294])


In [13]:
# Prepare node features
# One-hot encode locality type, aggregated by store_id
locality_encoded = weekly_data.groupby('store_id')['locality_type'].first()  # Take first value per store
locality_encoded = pd.get_dummies(locality_encoded, prefix='locality_type')  # One-hot encode

# Convert locality type to a tensor
locality_tensor = torch.tensor(locality_encoded.values, dtype=torch.float)

# Prepare time-series node features
node_features = torch.tensor(time_series.values, dtype=torch.float)

# Concatenate locality type with time-series features
node_features = torch.cat([node_features, locality_tensor], dim=1)

# Debug: Print updated node features shape
print(f"Updated node features shape: {node_features.shape}")

print(f"time_series shape: {time_series.shape}")
print(f"locality_encoded shape: {locality_encoded.shape}")



Updated node features shape: torch.Size([100, 208])
time_series shape: (100, 205)
locality_encoded shape: (100, 3)


In [19]:
# Define the STGNN model class
class STGNN(nn.Module):
    def __init__(self, in_channels, spatial_out, temporal_out, time_steps, forecast_steps):
        super(STGNN, self).__init__()
        self.gcn = GCNConv(in_channels, spatial_out)
        self.temporal_conv = nn.Conv1d(spatial_out, temporal_out, kernel_size=3, padding=1)
        
        self.flatten_out_size = None
        self.forecast_steps = forecast_steps

    def forward(self, x, edge_index, edge_weight):
        # Spatial convolution (GCN)
        x = self.gcn(x, edge_index, edge_weight)
        x = torch.relu(x)

        # Ensure input is float32
        x = x.to(torch.float32)

        # Get dimensions dynamically
        num_nodes, spatial_out = x.size(0), x.size(1)
        sequence_length = x.numel() // (num_nodes * spatial_out)  # Infer sequence length

        # Temporal convolution
        x = x.view(num_nodes, spatial_out, sequence_length).permute(0, 2, 1)  # (batch, seq_len, spatial_out)
        # Check shape after reshaping
        print(f"x shape after reshaping: {x.shape}")
        x = self.temporal_conv(x)
        x = torch.relu(x)

        # Flatten dynamically
        x = x.view(x.size(0), -1)  # Flatten: (batch, flattened_channels)

        if self.flatten_out_size is None:
            self.flatten_out_size = x.size(1)
            self.fc = nn.Linear(self.flatten_out_size, top_n)

        # Pass through fully connected layer
        x = self.fc(x)
        return x


In [20]:
# Define the TimeSeriesDataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, time_steps, forecast_steps):
        self.data = data
        self.time_steps = time_steps
        self.forecast_steps = forecast_steps

    def __len__(self):
        return self.data.shape[1] - self.time_steps - self.forecast_steps

    def __getitem__(self, idx):
        x = self.data[:, idx:idx + self.time_steps]
        y = self.data[:, idx + self.time_steps:idx + self.time_steps + self.forecast_steps]
        return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.float)


In [21]:
# Hyperparameters and dataset preparation
time_steps = 8
forecast_steps = 1
spatial_out = 32
temporal_out = 64
epochs = 50
batch_size = 16
learning_rate = 0.001

# Prepare dataset and dataloader
num_nodes = node_features.shape[0]  # Dynamically determine the number of nodes
dataset = TimeSeriesDataset(node_features, time_steps, forecast_steps)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define edge_index and edge_weight (for graph structure)
edge_index = torch.combinations(torch.arange(num_nodes), r=2).t().contiguous()  # Fully connected graph
edge_weight = torch.ones(edge_index.size(1))  # Equal weights for edges

# Debug: Print the dataloader batch example
for x, y in dataloader:
    print(f"Example batch x shape: {x.shape}")
    print(f"Example batch y shape: {y.shape}")
    break


Example batch x shape: torch.Size([16, 100, 8])
Example batch y shape: torch.Size([16, 100, 1])


C:\Users\rznis\AppData\Local\Temp\ipykernel_10404\4150409090.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.float)


In [22]:
# Initialize the model
model = STGNN(
    in_channels=node_features.shape[1],  # Set number of input channels (features per node)
    spatial_out=spatial_out,
    temporal_out=temporal_out,
    time_steps=time_steps,
    forecast_steps=forecast_steps
)

print(f"Node features shape: {node_features.shape}")  # Shape of input features
print(f"Edge index shape: {edge_index.shape}, Edge weight shape: {edge_weight.shape}")  # Graph structure shapes

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the Model
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        optimizer.zero_grad()

        # Forward pass
        out = model(x, edge_index, edge_weight)

        # Ensure the target shape is the same as the output shape
        y = y.squeeze(-1)  # Remove the last dimension (from shape [16, 970, 1] to [16, 970])

        # Print the shape of y before reshape to debug
        print(f"Shape of y before reshape: {y.shape}")

        # Now, instead of reshaping y to [16, 16], reshape y dynamically to match the output shape
        y = y.view(y.size(0), -1)  # Flatten y dynamically (automatically determines the correct size)
        
        # Print the shape of y after reshape
        print(f"Shape of y after reshape: {y.shape}")

        # Ensure that the model output and target (y) have the same shape
        # If out is of shape (batch_size, num_nodes, forecast_steps), then y should also be of shape (batch_size, num_nodes, forecast_steps)
        print(f"Shape of model output out: {out.shape}")

        # Ensure y matches out's shape by verifying that both are [batch_size, num_nodes, forecast_steps]
        if out.shape != y.shape:
            raise ValueError(f"Shape mismatch: out shape {out.shape} does not match y shape {y.shape}")

        # Now calculate the loss
        loss = criterion(out, y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")


Node features shape: torch.Size([100, 208])
Edge index shape: torch.Size([2, 4950]), Edge weight shape: torch.Size([4950])


C:\Users\rznis\AppData\Local\Temp\ipykernel_10404\4150409090.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.float)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1600x8 and 208x32)

In [ ]:
# Forecast Future Demand
model.eval()
with torch.no_grad():
    # Get the most recent time window (last `time_steps` weeks)
    x = node_features[:, -time_steps:]  # Shape: [num_stores, time_steps]
    
    # Add batch dimension: Shape becomes [1, num_stores, time_steps]
    x = x.unsqueeze(0)
    
    # Permute to match expected input shape: [1, time_steps, num_stores]
    x = x.permute(0, 2, 1)  # Shape becomes [1, time_steps, num_stores] (for spatial convolution)

    # Ensure edge_index matches the correct number of nodes (stores)
    num_stores = x.size(2)  # Get number of stores from x (it should be the same as node_features)
    
    # Fix edge_index generation: Generate fully connected edges for the given stores
    edge_index = torch.combinations(torch.arange(num_stores), r=2).t().contiguous()  # Fully connected graph
    
    # Ensure edge_weight matches the edge_index size
    edge_weight = torch.ones(edge_index.size(1))  # Equal weights for edges

    # Pass through the model
    prediction = model(x, edge_index, edge_weight)
    print("Predicted Demand:", prediction)


RuntimeError: index 8 is out of bounds for dimension 0 with size 8